# Automated Project: Planning, Estimation, and Allocation 🚀

## Introduction
This notebook demonstrates an `automated approach to project planning, estimation, and task allocation using AI agents`. The objective is to develop a website for a small business by leveraging AI-driven project management techniques.

## Initial Imports
For more info see [.env.example](https://github.com/micag2025/DeepLearningAI-CrewAI/blob/main/Practical_Multi_AI_Agents_and_Advanced_use_cases_with_CrewAI/Automated_project_planning_estimation_allocation/.env.example) and [helper.py](https://github.com/micag2025/DeepLearningAI-CrewAI/blob/main/Practical_Multi_AI_Agents_and_Advanced_use_cases_with_CrewAI/Automated_project_planning_estimation_allocation/helper.py)

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Set OpenAI Model

In [ ]:
# Uncomment the desired model
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
#os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
#os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo'


## Loading Tasks and Agents YAML files

In [5]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Create Pydantic Models for Structured Output
Define Pydantic models to structure the output of tasks and project plans.

In [ ]:
from typing import List
from pydantic import BaseModel, Field

# Define a model for task estimates
class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

# Define a model for project milestones
class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

# Define a model for the overall project plan
class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

## Create Crew, Agents and Tasks
Instantiate the crew, agents, and tasks based on the loaded configurations.

In [7]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

## Crew's Inputs
Define the input parameters for the crew, including project type, objectives, industry, team members, and requirements.

In [ ]:
from IPython.display import display, Markdown

# Define the project inputs
project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



## Kicking off the crew
Run the crew with the defined inputs and execute the tasks.

In [9]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

 [DEBUG]: == Working Agent: The Ultimate Project Planner

 [INFO]: == Starting Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to sh

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [10]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics['prompt_tokens'] + crew.usage_metrics['completion_tokens']) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics])
df_usage_metrics

Total costs: $0.0025


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,16956,8967,7989,9


### Result
Display the result of the crew execution.

In [11]:
result.dict()

{'tasks': [{'task_name': 'Create a Responsive Design',
   'estimated_time_hours': 80.0,
   'required_resources': ['Bob Smith (Designer)']},
  {'task_name': 'Develop a Visually Appealing UI',
   'estimated_time_hours': 120.0,
   'required_resources': ['Bob Smith (Designer)']},
  {'task_name': 'Develop User-Friendly Navigation',
   'estimated_time_hours': 40.0,
   'required_resources': ['Jane Doe (Software Eng.)']},
  {'task_name': 'About Us Page',
   'estimated_time_hours': 40.0,
   'required_resources': ['Jane Doe (Software Eng.)', 'Bob Smith (Designer)']},
  {'task_name': 'Services Page',
   'estimated_time_hours': 60.0,
   'required_resources': ['Jane Doe (Software Eng.)', 'Bob Smith (Designer)']},
  {'task_name': 'Contact Us Page',
   'estimated_time_hours': 40.0,
   'required_resources': ['Jane Doe (Software Eng.)']},
  {'task_name': 'Blog Section',
   'estimated_time_hours': 60.0,
   'required_resources': ['Jane Doe (Software Eng.)']},
  {'task_name': 'SEO Optimization',
   'estim

## Further Inspection
Inspect the task details in more depth.

In [ ]:
# Convert the tasks to a DataFrame for better inspection
tasks = result.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Create a Responsive Design,80.000000,['Bob Smith (Designer)']
1,Develop a Visually Appealing UI,120.000000,['Bob Smith (Designer)']
2,Develop User-Friendly Navigation,40.000000,['Jane Doe (Software Eng.)']
3,About Us Page,40.000000,"['Jane Doe (Software Eng.)', 'Bob Smith (Designer)']"
4,Services Page,60.000000,"['Jane Doe (Software Eng.)', 'Bob Smith (Designer)']"
5,Contact Us Page,40.000000,['Jane Doe (Software Eng.)']
6,Blog Section,60.000000,['Jane Doe (Software Eng.)']
7,SEO Optimization,80.000000,['Jane Doe (Software Eng.)']
8,Social Media Integration,40.000000,['Jane Doe (Software Eng.)']
9,Testimonials Section,40.000000,['Bob Smith (Designer)']


### Inspecting Milestones

In [13]:
milestones = result.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Milestone Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Design Complete,"['Create a Responsive Design', 'Develop a Visually Appealing UI']"
1,Core Functionality Rollout,"['Develop User-Friendly Navigation', 'About Us Page', 'Services Page', 'Contact Us Page', 'Blog Section', 'SEO Optimization', 'Social Media Integration']"
2,Final Touches,['Testimonials Section']
3,Quality Assurance,['Quality Assurance']


### Using another model: HuggingFace 

In [ ]:
#### Hugging Face (HuggingFaceHub endpoint)
hg_api_key = os.environ.get("HF_API_KEY")  # Retrieves the API key

from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token=hg_api_key,
    task="text-generation",
)

### you will pass "llm" to your agent function

c:\Users\Michela\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


- Loading Tasks and Agents YAML files

In [15]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

- Create Pydantic Models for Structured Output

In [16]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

- Create Crew, Agents and Tasks

In [17]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

2025-02-20 13:05:12,948 - 3044 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


- Crew's Inputs


In [18]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



- Kicking off the crew

In [19]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

 [DEBUG]: == Working Agent: The Ultimate Project Planner

 [INFO]: == Starting Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to sh

- Usage Metrics and Costs

In [20]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics['prompt_tokens'] + crew.usage_metrics['completion_tokens']) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics])
df_usage_metrics

Total costs: $0.0022


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,14424,7866,6558,9


In [21]:
result.dict()

{'tasks': [{'task_name': 'Project Initiation and Planning',
   'estimated_time_hours': 40.0,
   'required_resources': ['John Doe (Project Manager)']},
  {'task_name': 'Responsive Design Implementation',
   'estimated_time_hours': 80.0,
   'required_resources': ['Bob Smith (Designer)']},
  {'task_name': 'User Interface Development',
   'estimated_time_hours': 120.0,
   'required_resources': ['Bob Smith (Designer)']},
  {'task_name': 'Core Website Development',
   'estimated_time_hours': 160.0,
   'required_resources': ['Jane Doe (Software Engineer)']},
  {'task_name': 'Advanced Features Implementation',
   'estimated_time_hours': 120.0,
   'required_resources': ['Jane Doe (Software Engineer)']},
  {'task_name': 'Content Development',
   'estimated_time_hours': 80.0,
   'required_resources': ['John Doe', 'Jane Doe']},
  {'task_name': 'Testing and Quality Assurance',
   'estimated_time_hours': 80.0,
   'required_resources': ['Alice Johnson', 'Tom Brown']},
  {'task_name': 'SEO and Perform

In [22]:
tasks = result.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Project Initiation and Planning,40.000000,['John Doe (Project Manager)']
1,Responsive Design Implementation,80.000000,['Bob Smith (Designer)']
2,User Interface Development,120.000000,['Bob Smith (Designer)']
3,Core Website Development,160.000000,['Jane Doe (Software Engineer)']
4,Advanced Features Implementation,120.000000,['Jane Doe (Software Engineer)']
5,Content Development,80.000000,"['John Doe', 'Jane Doe']"
6,Testing and Quality Assurance,80.000000,"['Alice Johnson', 'Tom Brown']"
7,SEO and Performance Optimization,40.000000,['Jane Doe (Software Engineer)']
8,Social Media Integration,40.000000,['Jane Doe (Software Engineer)']
9,Project Closure,40.000000,['John Doe (Project Manager)']


In [23]:
milestones = result.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Milestone Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Project Start,['Project Initiation and Planning']
1,Design Phase,"['Responsive Design Implementation', 'User Interface Development']"
2,Development Phase,"['Core Website Development', 'Advanced Features Implementation']"
3,Content and Testing,"['Content Development', 'Testing and Quality Assurance']"
4,Optimization and Integration,"['SEO and Performance Optimization', 'Social Media Integration']"
5,Project End,['Project Closure']


## Conclusion
This notebook provided an `automated approach to project planning, estimation, and task allocation using AI agents`. The AI-driven method enabled efficient project management, ensuring that all aspects of the website development were thoroughly planned and executed.

## Extra practice
Now that you have learned how to build and run a multi-agent system for project planning, estimation, and task allocation using AI agents, try it yourself! Here are some suggestions on what you can do:

- **Modify the input parameters**: Change the project type, objectives, industry, team members, and requirements to see how the system adapts to different projects.  
- **Explore different OpenAI models**: Uncomment and use different OpenAI models to compare their performance.  
- **Adjust task and agent configurations**: Tweak the YAML configurations for tasks and agents to understand their impact on the overall project plan.  
- **Analyze the output**: Review the generated project plan, task estimates, and resource allocation to ensure they meet your expectations.